# Importing corpus and reading it

In [128]:
# FULL DATASET 

from nltk.corpus import brown
train_lab = brown.tagged_sents()[0:len(brown.tagged_sents())*20/100]
# test_lab = brown.tagged_sents()[len(brown.tagged_sents())*2/10+1:len(brown.tagged_sents())]
# train = brown.sents()[0:len(brown.sents())*8/10]
#test = brown.sents()[len(brown.sents())*80/100+1:len(brown.sents())][10:12]
test = brown.sents()[0:len(brown.sents())*20/100][10:12]
print test

[[u'It', u'urged', u'that', u'the', u'city', u'``', u'take', u'steps', u'to', u'remedy', u"''", u'this', u'problem', u'.'], [u'Implementation', u'of', u"Georgia's", u'automobile', u'title', u'law', u'was', u'also', u'recommended', u'by', u'the', u'outgoing', u'jury', u'.']]


# Making the Data Structures

In [150]:
from itertools import tee, izip

transition_tri={}
transition_bi={}
context={}
emission={}

def window(iterable, size):
    iters = tee(iterable, size)
    for i in xrange(1, size):
        for each in iters[i:]:
            next(each, None)
    return izip(*iters)


# MAKING DICT {(TAG,TAG,TAG) : COUNT}
           

for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,3):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append(temp[1][1])
        temp2.append(temp[2][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if transition_tri.has_key(tuple(temp)):
            transition_tri[tuple(temp)]+=1
        else:
            transition_tri[tuple(temp)]=1


            
# MAKING DICT {(TAG,TAG) : COUNT}


for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,2):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append(temp[1][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if transition_bi.has_key(tuple(temp)):
            transition_bi[tuple(temp)]+=1
        else:
            transition_bi[tuple(temp)]=1
            

            
# MAKING CONTEXT {(TAG) : COUNT}


for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,1):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if context.has_key(tuple(temp)):
            context[tuple(temp)]+=1
        else:
            context[tuple(temp)]=1


# MAKING EMISSION { (TAG,WORD) : COUNT }

N=0

for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,1):
        N+=1
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append((temp[0][0]).lower())
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if emission.has_key(tuple(temp)):
            emission[tuple(temp)]+=1
        else:
            emission[tuple(temp)]=1
        
       
print context

SyntaxError: invalid syntax (<ipython-input-150-23263e99d1d4>, line 112)

# Interpolation and Transition Probability Calculation

In [154]:
lam_1 = 0.33
lam_2 = 0.33
lam_3 = 0.33

def Transition_Prob(trig):
    if len(trig) > 2:                      # IT IS A TRIGRAM
        
        prev1 = trig[0]
        prev2 = trig[1]
        tag = trig[2]
        
        P1 = lam_1 * context[tuple([prev1])]/N + (1-lam_1)/N
        
        P2 = lam_2 * transition_bi.get(tuple((prev1+" "+prev2).split()),0)/(context[tuple([prev1])]+1)+(1-lam_2)*P1
        
        P3 = lam_3*transition_tri.get(tuple((prev1+" "+prev2+" "+tag).split()),0)/(transition_bi.get(tuple((prev1+" "+prev2).split()),0)+1)+(1-lam_3)*P2
        return P3
    else:                                            # ITS A BIGRAM
        prev = trig[0]
        tag  = trig[1]
        
        P1 = lam_1 * context[tuple([prev])]/N + (1-lam_1)/N
        
        P2 = lam_2 * transition_bi.get(tuple((prev+" "+tag).split()),0)/(context[tuple([prev])]+1)+(1-lam_2)*P1
        return P2


#    Emission Probability with some smoothing probability

In [155]:
lamda = 0.33

def Emission_Prob(tag2word):
    
    tag = tag2word[0]
    word = tag2word[1]
    
    prob = lamda * emission.get(tag2word,0)/context[tuple([tag])]+(1-lamda)/N
    
    return prob

# Forward Step

In [153]:
import math

def Fetch_Context(k):
    if k == -1 or k == 0:               # In case we are at beginning retun start
        return [tuple(["<s>"])];
    else:                               # For a word return full sentence
        return context


def forward_step(line):
    
    words=[]
    
    for i in line:
        words.append(i.lower())
    
    # NOW WE HAVE A LIST OF WORDS FOR A SENTENCE, ALL IN LOWER CASE
    
    l=len(words)
    
    best_score={}
    best_edge={}
    
    best_score[(0,"<s>","<s>")]=0     # init
    best_edge[(0,"<s>","<s>")]=None   # init
    
    for k in range(1,l+1):        # Travel in a sentence  
        
        for u in Fetch_Context(k-1):
            for v in Fetch_Context(k):
                for w in Fetch_Context(k-2):
                    
                    tri=tuple([w[0],u[0],v[0]])
                    if emission.get(tuple((v[0]+" "+words[k-1]).split()),0)!=0:
                        
                        score=best_score.get(tuple([k-1,w[0],u[0]]),0)+(-math.log(Transition_Prob(tri)))+(-math.log(Emission_Prob(tuple([v[0],words[k-1]]))))
                        
                        if best_score.get(tuple([k,u[0],v[0]]),0) == 0 or best_score.get(tuple([k,u[0],v[0]]),0) > score:
                            
                            best_score[tuple([k,u[0],v[0]])]=score
                            best_edge[tuple([k,u[0],v[0]])]=w[0]

    
    # NOW FOR 
    
    max_score = float('Inf')
    
    u_max = None
    v_max = None
    
    for u in S(l-1):
        for v in S(l):
            
            tri=tuple([u[0],v[0],"</s>"])
            
            score=best_score.get(tuple([l,u[0],v[0]]),0)+(-math.log(Transition_Prob(tri)))
            
            if score < max_score:
                max_score = score
                u_max = u[0]
                v_max = v[0]
    
    tags = []
    tags.append(v_max)
    tags.append(u_max)
    
    for i, k in enumerate(range(l-2, 0, -1)):
        tags.append(best_edge[tuple([k+2,tags[i+1],tags[i]])])
    tags.reverse()

    tagged_sentence = []
    for j in range(0, l):
        tagged_sentence.append((words[j],tags[j]))
    
    
    return tagged_sentence

#print test[1]
forward_step(test[0])

0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00741243929619
0.00741243929619
0.00922457636132
0.0134283980174
0.00625575606313
0.00625575606313
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00621719995536
0.00910890803802
0.0112943385016
0.00627503411701
0.00627503411701
0.0203618245957
0.0269618245957
0.0237450382828
0.0963376308754
0.00639070244032
0.00639070244032
0.00621719995536
0.00621719995536
0.0184629613709
0.0607876106695
0.0135516307987
0.0121592257354
0.00621719995536
0.00621719995536
0.00750882956561
0.022067653095
0.0234690440569
0.00696904405685
0.0112495976853
0.00885829333752
0.00633286827867
0.00633286827867
0.00621719995536
0.00621719995536
0.0064099804942
0.0064099804942
0.0204384462301
0.0189407155191
0.0223170467473
0.00660276103305
0.00745099540396
0.00745099540396
0.0310028053406
0.0231456624835
0.00712326848793
0.00712326848793
0

2.7118358849e-05
3.47069002108e-05
2.18458008899e-06
0.000268325566419
0.00124236905407
1.68196241438e-05
5.09680602716e-05
0.000109508236491
3.26865742638e-06
0.000247728097008
2.92865135238e-05
0.000216289854224
0.00161529165813
0.000283502649142
4.89477343247e-06
0.00148616656876
0.00215516217215
0.000417928238979
0.000345837096042
0.000284044687811
0.00187004983242
1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.0041542007823
0.000279708378461
0.000377275338827
0.0201638549009
0.00174592297728
0.00328693891239
2.11559334934e-05
0.00245870382662
8.45744577308e-05
0.000759412600258
0.00387396679059
6.93973750073e-05
0.000488935304578
1.84457401499e-05
0.000964303217025
0.00735656523696
0.000611436043704
0.00701616495302
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00049218753659
0.000490019381916
8.94528057491e-05
0.00621719995536
0.000281334494468
0.000150161136643
4.35273476377e-06
0.00153452789649
0.000561026447515
7.60496677595e-06
4.98839829343

0.00238751429659
0.00680454234057
2.27820494994e-05
0.000643416325157
0.000407087465605
0.0119340817941
1.30253534629e-05
2.72661875768e-06
4.89477343247e-06
0.000152329291318
0.000117096777853
0.000298679731866
0.00113558743633
2.18458008899e-06
0.00372924246604
0.000847764903255
0.000224962472923
3.81069609508e-06
0.00095702785754
1.30253534629e-05
0.000785435280047
3.19967068673e-05
0.000352341560067
0.000262905179732
2.72661875768e-06
0.00185914074712
0.000529046166062
2.92865135238e-05
2.98285521925e-05
0.00825255515631
0.0144477618506
1.73616628125e-05
8.45744577308e-05
3.85011708916e-05
1.03151601194e-05
0.000345837096042
0.00164889805559
2.7118358849e-05
3.47069002108e-05
2.18458008899e-06
0.000268325566419
0.00124236905407
1.68196241438e-05
5.09680602716e-05
0.000109508236491
3.26865742638e-06
0.000247728097008
2.92865135238e-05
0.000216289854224
0.00161529165813
0.000283502649142
4.89477343247e-06
0.00176242646048
0.00215516217215
0.000417928238979
0.000345837096042
0.0002840

2.92865135238e-05
2.98285521925e-05
0.0082670782777
0.0143727137646
1.73616628125e-05
8.45744577308e-05
3.85011708916e-05
1.03151601194e-05
0.000345837096042
0.00164889805559
2.7118358849e-05
3.47069002108e-05
2.18458008899e-06
0.000268325566419
0.00143538520422
1.68196241438e-05
5.09680602716e-05
0.00120950823649
3.26865742638e-06
0.000247728097008
2.92865135238e-05
0.000216289854224
0.00161529165813
0.000283502649142
4.89477343247e-06
0.00130199330762
0.00232203009668
0.000417928238979
0.000345837096042
0.000284044687811
0.00187004983242
1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.00527946503912
0.000279708378461
0.000377275338827
0.0202946164805
0.00174592297728
0.00335987310338
2.11559334934e-05
0.00543270603169
8.45744577308e-05
0.000759412600258
0.0040905494942
6.93973750073e-05
0.00294287537117
1.84457401499e-05
0.000964303217025
0.0074217336107
0.000611436043704
0.00711866051046
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00049218753659
0

4.89477343247e-06
0.00130199330762
0.00221078481366
0.000417928238979
0.000692932857424
0.000284044687811
0.00193415537025
1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.00424075957129
0.000279708378461
0.000377275338827
0.0203540535622
0.00174592297728
0.00339634019888
2.11559334934e-05
0.00265372036466
8.45744577308e-05
0.000759412600258
0.00396678794928
6.93973750073e-05
0.000488935304578
1.84457401499e-05
0.000964303217025
0.00771499129252
0.000611436043704
0.00706741273174
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00049218753659
0.000490019381916
8.94528057491e-05
0.00639070244032
0.000281334494468
0.000150161136643
4.35273476377e-06
0.00153452789649
0.000561026447515
7.60496677595e-06
4.98839829343e-05
4.5005634916e-05
8.89107670804e-05
0.0184831965384
1.41094308003e-05
0.000153413368655
0.00621719995536
0.00123248974782
0.000289465074498
7.69859163691e-05
5.7472524296e-05
0.00238751429659
0.0068751813182
2.27820494994e-05
0.000643416325157


...

0.00104002861881
0.00535708484077
0.0358459848861
1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.0288234556433
0.00285563070856
0.00101353433163
0.021578457444
0.00174592297728
0.015284613331
2.11559334934e-05
0.0641814381155
0.00151102607063
0.026817626886
0.0480258979409
6.93973750073e-05
0.0451506445166
1.84457401499e-05
0.00282960130476
0.00813858572182
0.000807620604484
0.00896358054446
0.00127327454249
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.0290133562246
0.0213023117407
0.0189638430497
0.0444841369159
0.022049867313
0.000150161136643
4.35273476377e-06
0.00325332648307
0.0304971966603
0.0170152972745
0.00733889497195
0.210359639781
0.0150097696628
0.00219500797906
0.0088581094308
0.0197544771984
0.00621719995536
0.0258126102297
0.0301566320539
0.0204620922993
5.7472524296e-05
0.044308772129
0.0288262436185
2.27820494994e-05
0.0241329610132
0.00423948746561
0.0119340817941
0.0288521557882
2.72661875768e-06
4.89477343247e-06
0.000941972148461
0.0062873293359

1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.0041542007823
0.000279708378461
0.000377275338827
0.0201816860254
0.00174592297728
0.00328693891239
2.11559334934e-05
0.00245870382662
8.45744577308e-05
0.000759412600258
0.00387396679059
6.93973750073e-05
0.000488935304578
1.84457401499e-05
0.000964303217025
0.00737285733039
0.000611436043704
0.00701616495302
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00049218753659
0.000490019381916
8.94528057491e-05
0.00621719995536
0.000281334494468
0.000150161136643
4.35273476377e-06
0.00153452789649
0.000561026447515
7.60496677595e-06
4.98839829343e-05
4.5005634916e-05
8.89107670804e-05
0.00102392747058
1.41094308003e-05
0.000153413368655
0.00621719995536
0.000990321073122
0.000289465074498
7.69859163691e-05
5.7472524296e-05
0.00233620308749
0.00680454234057
2.27820494994e-05
0.000643416325157
0.000407087465605
0.0119340817941
1.30253534629e-05
2.72661875768e-06
4.89477343247e-06
0.000152329291318
0.0001170967778

6.93973750073e-05
0.000488935304578
1.84457401499e-05
0.00146171604042
0.00779645175969
0.00571223462401
0.0105010139061
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00049218753659
0.000490019381916
0.00143762353746
0.00621719995536
0.000281334494468
0.000150161136643
4.35273476377e-06
0.00153452789649
0.000561026447515
7.60496677595e-06
4.98839829343e-05
4.5005634916e-05
8.89107670804e-05
0.00102392747058
1.41094308003e-05
0.000153413368655
0.00621719995536
0.00111140541047
0.000289465074498
7.69859163691e-05
5.7472524296e-05
0.00249013671479
0.00719305671757
2.27820494994e-05
0.00213481598789
0.000996687465605
0.0119340817941
1.30253534629e-05
2.72661875768e-06
4.89477343247e-06
0.000152329291318
0.000117096777853
0.000298679731866
0.00113558743633
2.18458008899e-06
0.00411493806133
0.00141359983608
0.000224962472923
3.81069609508e-06
0.000808738119109
1.30253534629e-05
0.000577829646244
0.0076561346379
0.00103369749227
0.00163335972519
2.72661875768e-06
0

1.24833147942e-05
1.64254142029e-06
0.00630717837436
0.0041542007823
0.000279708378461
0.000377275338827
0.0201757423172
0.00174592297728
0.00569376721521
2.11559334934e-05
0.00260496623015
8.45744577308e-05
0.000759412600258
0.0193524066904
6.93973750073e-05
0.000488935304578
1.84457401499e-05
0.00133736283457
0.00735656523696
0.000611436043704
0.00701616495302
0.00104344086265
2.18458008899e-06
1.13992374568e-05
1.64254142029e-06
0.00219858224442
0.000734869880254
0.00278579426916
0.00621719995536
0.000281334494468
0.000150161136643
4.35273476377e-06
0.00161265510497
0.000561026447515
7.60496677595e-06
4.98839829343e-05
4.5005634916e-05
8.89107670804e-05
0.00102392747058
1.41094308003e-05
0.0103459665601
0.00621719995536
0.00123248974782
0.000289465074498
0.00478124123552
5.7472524296e-05
0.0144134458409
0.00678688259616
2.27820494994e-05
0.000643416325157
0.000407087465605
0.0119340817941
1.30253534629e-05
2.72661875768e-06
4.89477343247e-06
0.000152329291318
0.000117096777853
0.000

KeyboardInterrupt: 